- Used environment: `PaperGPT` (in `environment.yml`)

In [20]:
!conda install -c conda-forge openai -y

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Retrieving notices: ...working... done
done
Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 22.11.1
  latest version: 23.9.0

Please update conda by running

    $ conda update -n base -c conda-forge conda

Or to minimize the number of packages updated during conda update use

     conda install conda=23.9.0



## Package Plan ##

  environment location: /users/cwoest/Applications/anaconda3/envs/PaperGPT

  added / updated specs:
    - openai


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    click-8.1.7                |un

In [35]:
from PyPDF2 import PdfReader

import yaml
import os
import openai

## Read the raw text from the file

First we need to extract the raw text from the file containing the paper.

In [15]:
# Read the file if it's a PDF.
reader = PdfReader("../tests/attention_is_all_you_need.pdf")
number_of_pages = len(reader.pages)
page = reader.pages[0]
text = page.extract_text()
print(f"Number of pages: {number_of_pages}")
print(f"1st page: {page}")
print(f"1st page, number of chars: {len(text)}")

Number of pages: 15
1st page: {'/Type': '/Page', '/Contents': [IndirectObject(101, 0, 5156558800), IndirectObject(5956, 0, 5156558800)], '/MediaBox': [0, 0, 612, 792], '/Parent': IndirectObject(109, 0, 5156558800), '/Resources': IndirectObject(100, 0, 5156558800)}
1st page, number of chars: 2853


In [16]:
N = 100
print(f"First {N} chars: \n    {text[:N]}\n")
print(f"Last  {N} chars: \n    {text[-N:]}")

First 100 chars: 
    Provided proper attribution is provided, Google hereby grants permission to
reproduce the tables and

Last  100 chars: 
    ormation Processing Systems (NIPS 2017), Long Beach, CA, USA.arXiv:1706.03762v7  [cs.CL]  2 Aug 2023


In [17]:
raw_text_per_page = {}

# Save the raw text as value under the page_id key.
for page_id in range(len(reader.pages)):
    page = reader.pages[page_id]
    raw_text_in_the_page = page.extract_text()
    raw_text_per_page[page_id] = raw_text_in_the_page

In [18]:
raw_text_per_page.get(1)

'1 Introduction\nRecurrent neural networks, long short-term memory [ 13] and gated recurrent [ 7] neural networks\nin particular, have been firmly established as state of the art approaches in sequence modeling and\ntransduction problems such as language modeling and machine translation [ 35,2,5]. Numerous\nefforts have since continued to push the boundaries of recurrent language models and encoder-decoder\narchitectures [38, 24, 15].\nRecurrent models typically factor computation along the symbol positions of the input and output\nsequences. Aligning the positions to steps in computation time, they generate a sequence of hidden\nstates ht, as a function of the previous hidden state ht−1and the input for position t. This inherently\nsequential nature precludes parallelization within training examples, which becomes critical at longer\nsequence lengths, as memory constraints limit batching across examples. Recent work has achieved\nsignificant improvements in computational efficiency th

## Preprocess the text

Before we feed the LLM with the papers information we need to clean the extracted text. For that we need to perform some processing steps

- [ ] Remove all unncessary and un-informative chars 
- [ ] Select which parts/pages contains which sections
- [ ] Select only the relevant text for the main summarization
- [ ] Create a dictionary with keys stating the section and values contains the cleaned text per section

### BART

https://huggingface.co/facebook/bart-large-cnn

In [ ]:
from transformers import pipeline

summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

In [42]:
len(list(raw_text_per_page.keys()))

15

In [44]:
summarizer(raw_text_per_page.get(page_id),
        max_length=130, 
        min_length=30, 
        do_sample=False
    )

IndexError: index out of range in self

In [45]:
print(f"Summaries per Page")
print(f"==================")

summaries_per_page = {}

# for page_id in range(len(list(raw_text_per_page.keys()))):
for page_id in range(5):
    print(f"{4*' '}Page {page_id}")
    text = raw_text_per_page.get(page_id)

    summary = summarizer(text, 
        max_length=130, 
        min_length=30, 
        do_sample=False
    )

    summaries_per_page[page_id] = summary

Summaries per Page
    Page 0
    Page 1
    Page 2
    Page 3
    Page 4


In [46]:
summaries_per_page.get(0)

[{'summary_text': 'The dominant sequence transduction models are based on complex recurrent orconvolutional neural networks. We propose a new simple network architecture, the Transformer, based solely on attention mechanisms. The Transformer is superior in quality while being more parallelizable and requiring significantly less time to train.'}]

In [47]:
summaries_per_page.get(1)

[{'summary_text': 'The Transformer is the first transduction model relying solely on self-attention to compute representations of its input and output. It can reach a new state of the art in machine translation quality after being trained for as little as twelve hours on eight P100 GPUs. In the following sections, we will describe the Transformer, motivate patrioticself-att attention and discuss its advantages over models such as [17, 18] and [9].'}]

### pygmalion-6b

In [50]:
from transformers import pipeline, Conversation

pipe = pipeline("conversational", model="PygmalionAI/pygmalion-6b")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [52]:
from transformers import Conversation

conversation = Conversation(f"Can you summarize this in 1 sentence: {raw_text_per_page.get(0)}")
conversation = pipe(conversation)

Conversation input is to long (711), trimming it to (20 - 10)
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


KeyboardInterrupt: 

### GPT

In [ ]:
from transformers import pipeline, set_seed
generator = pipeline('text-generation', model='gpt2-large')
set_seed(42)
generator("Hello, I'm a language model,", max_length=30, num_return_sequences=5)

In [49]:
from transformers import pipeline, set_seed
generator = pipeline('text-generation', model='gpt2-large')
set_seed(42)
generator("Hello, I'm a language model,", max_length=30, num_return_sequences=5)


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': "Hello, I'm a language model, but what I'm really doing is making a human-readable version of something with a deep, powerful and expressive"},
 {'generated_text': "Hello, I'm a language model, this is my first commit and I'm going to put it on github. I've also put my code in"},
 {'generated_text': "Hello, I'm a language model, and you'll hear me in this chapter!\n\nOne of the features we love is the ability to have"},
 {'generated_text': "Hello, I'm a language model, not a developer. If you don't know my code, please don't comment anything on the repo, because"},
 {'generated_text': "Hello, I'm a language model, I'm not a language designer. That should probably be a rule not a rule. I am speaking as a"}]

In [31]:
# Load the YAML configuration file
with open("../config/config_private.yml", "r") as config_file:
    config_data = yaml.safe_load(config_file)

# Extract the API key from the loaded data
api_key = config_data["OPENAI_API_KEY"]

In [32]:


openai.api_key = api_key

In [34]:
model_engine = "text-davinci-002"
temperature = 0

max_tokens = 1000

completions = openai.Completion.create(
    engine=model_engine,
    prompt='Hallo Welt',
    max_tokens=max_tokens,
    temperature=temperature
)

# Get the summary from the first completion
summary = completions.choices[0].text
summary

RateLimitError: You exceeded your current quota, please check your plan and billing details.

### BERT

In [6]:
from transformers import BertTokenizer

# Load pre-trained tokenizer.
tokenizer = BertTokenizer.from_pretrained("bert-base-cased")
encoded_input = tokenizer("Do not meddle in the affairs of wizards, for they are subtle and quick to anger.")
encoded_input


{'input_ids': [101, 2091, 1136, 1143, 13002, 1107, 1103, 5707, 1104, 16678, 1116, 117, 1111, 1152, 1132, 11515, 1105, 3613, 1106, 4470, 119, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

### LLaMa

In [11]:
from transformers import pipeline

pipe = pipeline("text-generation", model="NousResearch/Llama-2-13b-hf")

KeyboardInterrupt: 

In [ ]:
pipe

In [41]:
tokenizer

BertTokenizerFast(name_or_path='bert-base-cased', vocab_size=28996, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True)

In [38]:
from transformers import BertConfig, BertModel

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.
